In [3]:
import pandas as pd
import numpy as np
import glob
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [4]:
method_1_files=glob.glob("metoda_1_*.csv")
method_2_files=glob.glob("metoda_2_*.csv")
method_3_files=glob.glob("metoda_3_*.csv")

def load_and_process(filepaths,method_label,has_center=False):
    dfs=[]
    for path in filepaths:
        df=pd.read_csv(path)
        if not has_center:
            df['center_x']=(df['x1']+df['x2'])/2
            df['center_y']=(df['y1']+df['y2'])/2
        df['distance_from_center']=np.sqrt(df['center_x']**2+df['center_y']**2)
        df['method']=method_label
        dfs.append(df)
    return pd.concat(dfs,ignore_index=True)

df1=load_and_process(method_1_files,method_label=0,has_center=False)
df2=load_and_process(method_2_files,method_label=1,has_center=True)
df3=load_and_process(method_3_files,method_label=2,has_center=True)
data=pd.concat([df1,df2,df3],ignore_index=True)

features=['x1','y1','x2','y2','length','center_x','center_y','distance_from_center']
X=data[features]
y=data['method']

scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
y_cat=tf.keras.utils.to_categorical(y,num_classes=3)

X_train,X_test,y_train,y_test=train_test_split(X_scaled,y_cat,test_size=0.2,random_state=42)

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32,activation='relu'),
    Dropout(0.2),
    Dense(3,activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history=model.fit(X_train,y_train, epochs=20,batch_size=32,validation_split=0.2,verbose=1)

loss,accuracy=model.evaluate(X_test,y_test,verbose=0)
print(f'Dokładność na danych testowych: {accuracy:.4f}')

y_pred_probs=model.predict(X_test)
y_pred=np.argmax(y_pred_probs,axis=1)
y_true=np.argmax(y_test,axis=1)

cm=confusion_matrix(y_true,y_pred)
disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Metoda 1','Metoda 2','Metoda 3'])
fig,ax=plt.subplots(figsize=(6,6))
disp.plot(ax=ax)
plt.title('Macierz pomyłek dla modelu sieć_neuronowa')
plt.savefig('macierz_pomyłek_dla_sieć_neuronowa.png',dpi=300)
plt.show()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [5]:
model.save("model_cieć_neuronowa.h5")
joblib.dump(scaler, "scaler_do_model_cieć_neuronowa.pkl")
print("Model i scaler zapisane.")

Model i scaler zapisane.
